<!-- ## Objectives

* Extract London Crime statistics CSV data from ??? and from the data folder into a data frame and explore its features
* Feature engeneering of interesting metadata and KPIs

## Inputs

* MPS Ward Level Crime (most recent 24 months).csv

## Outputs

* cleaned and feture engeneered csv data file 

## Additional Comments



We use the pandas and numpy libury for data manipulation and mathplotlib for basic visualisations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We load the csv file out of the data/raw folder into a pandas dataframe (df) and look up the shape and head.

In [ ]:
df = pd.read_csv("../data/raw/MPSCrime.csv")
print(df.shape)
df.head()

For easyer understanding we change the Major and Minor Text to Crime Category and Specific Crime Type repectivly.

In [ ]:
df = df.rename(columns={
    'MajorText': 'Crime Category',
    'MinorText': 'Specific Crime Type'
})

---

# Data Cleaning 

Checking for duplicates and missing values

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

Seems like there is no missing values or duplicated we should take care of.

Lets see the overall types.

In [ ]:
df.info()

### Unique data categories

lets see which categories of crime there are (MajorText) and which types are part of them.


In [ ]:
df['Crime Category'].unique()


In [ ]:
df['Specific Crime Type'].unique()

Lets group them for a better overview witch crime belngs in witch categorie

In [ ]:
crimes = df.groupby('Crime Category')['Specific Crime Type'].unique().reset_index()
crimes

other format:

In [ ]:
df[['Crime Category', 'Specific Crime Type']].drop_duplicates().sort_values(['Crime Category', 'Specific Crime Type'])

Now we want to see what a crime time series would look like.
First put the colums into real time series

In [ ]:
# Find all month columns
month_cols = [col for col in df.columns if col.isdigit()]

# Convert string YYYYMM → datetime
dates = pd.to_datetime(month_cols, format="%Y%m")

# Replace columns with datetime versions
df.rename(columns=dict(zip(month_cols, dates)), inplace=True)

Now we can look at the evolution of the crime categories

In [ ]:
major_pivot = df.pivot_table(
    index='Crime Category',
    values=[col for col in df.columns if isinstance(col, pd.Timestamp)],
    aggfunc='sum')

major_pivot.head()


In [ ]:
s = major_pivot.sum(axis=0)
plt.figure(figsize=(10, 4))
ax = s.plot(kind='bar')
ax.set_title('Total Crimes over time')
ax.set_xlabel('Year-Month')
ax.set_ylabel('Total Crimes')
ax.set_xticklabels([dt.strftime('%Y-%m') for dt in s.index], rotation=45)
plt.tight_layout()
plt.show()

The plot indicates that total crime levels show a slight decrease during colder months. We observe seasonal patterns with crime rates typically declining in winter periods (December-February) and increasing during warmer months (June-August). This temporal pattern aligns with our hypothesis testing, which confirmed a statistically significant 6% increase in summer crime compared to winter (p=0.029).

In [ ]:
# Create pivot table for specific crime types (minor categories)
minor_pivot = df.pivot_table(
    index='Specific Crime Type',
    values=[col for col in df.columns if isinstance(col, pd.Timestamp)],
    aggfunc='sum'
)

print(f"Minor pivot shape: {minor_pivot.shape}")
minor_pivot.head()

In [ ]:
# use a colormap with many distinct colors (tab20) and map it to the number of series
n = minor_pivot.shape[0]
cmap = plt.get_cmap('tab20')

# get list of colors from cmap (handle both ListedColormap and continuous cmap)
base_colors = getattr(cmap, "colors", [cmap(i / max(1, n - 1)) for i in range(n)])
colors = [base_colors[i % len(base_colors)] for i in range(n)]

ax = minor_pivot.T.plot(figsize=(12, 6), linewidth=2, color=colors)
ax.set_title('Crimes by Category over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Number of Crimes')
ax.tick_params(axis='x', rotation=45)
ax.legend(title='Crime Category', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Crime by Location

In [ ]:
loc_crime = df.groupby(['WardName', 'Crime Category'])[dates].sum().reset_index()
loc_crime.head()


In [ ]:
# Save the cleaned and processed dataset
df.to_csv('../data/clean/processed_crime_data.csv', index=False)
print("✓ Processed data saved to: data/clean/processed_crime_data.csv")
print(f"✓ Shape: {df.shape}")
print(f"✓ File size: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---